# Level 2 - Week 3 - 01 Retrieval as API

**Estimated time:** 60-90 minutes

## Learning Objectives

- Define /search contract
- Use typed request validation
- Keep response fields debuggable


## Overview

Retrieval should be a first-class endpoint.
Typed models make failures consistent.

## Practice Steps

- Implement SearchRequest and SearchResponse.
- Stub the search handler.


### Sample code

Minimal Pydantic models with constraints.


In [ ]:
from pydantic import BaseModel, Field
from typing import Dict, List

class SearchRequest(BaseModel):
    query: str = Field(min_length=1)
    top_k: int = Field(default=5, ge=1, le=50)
    filters: Dict | None = None

class SearchHit(BaseModel):
    doc_id: str
    chunk_id: str
    score: float
    text: str
    metadata: Dict

class SearchResponse(BaseModel):
    query: str
    hits: List[SearchHit]


### Student fill-in

Add a stub search handler that returns empty hits.


In [ ]:
def search_handler(req: SearchRequest) -> SearchResponse:
    # TODO: replace with real vector DB query
    return SearchResponse(query=req.query, hits=[])


## Self-check

- Do request fields have validation?
- Does response include chunk_id and metadata?


Legacy practice content from practice.ipynb

# Level 2 — Week 3 Practice: Retrieval API + Metrics

**Estimated time:** 60–90 minutes

## Learning Objectives

- Define retrieval API request/response contracts
- Return top-k hits with metadata and scores
- Add lightweight retrieval metrics for debugging
- Keep responses stable and testable


Legacy practice content from practice.ipynb

## Overview

This practice focuses on the **retrieval contract**. Your goal is to define
clear request/response models and add minimal metrics for debugging retrieval.

You will:

1. Define `/search` request/response schemas.
2. Create a stub retrieval function that returns top-k hits.
3. Add metrics (hit count, score stats) for quick inspection.

## Practice Steps

- Fill in the Pydantic models below.
- Implement a fake retrieval that returns deterministic data.
- Compute summary metrics and print them.


In [ ]:
# Legacy practice content
### Task 3.1: Retrieval API models

Define request/response models for search. Keep explicit types and defaults.


In [ ]:
# Legacy practice content
from pydantic import BaseModel
from typing import List, Dict, Optional

class SearchRequest(BaseModel):
    query: str
    top_k: int = 5
    filters: Optional[Dict] = None

class SearchHit(BaseModel):
    chunk_id: str
    doc_id: str
    score: float
    text: str
    metadata: Dict

class SearchResponse(BaseModel):
    hits: List[SearchHit]


Legacy practice content from practice.ipynb

### Task 3.2: Stub retrieval function

Create a deterministic retrieval function that returns top-k hits.
Keep it stable so you can write tests later.


In [ ]:
# Legacy practice content
from typing import List


def retrieve_stub(query: str, top_k: int = 5) -> List[SearchHit]:
    # TODO: replace with real vector search
    hits = []
    for i in range(top_k):
        hits.append(
            SearchHit(
                chunk_id=f"chunk-{i}",
                doc_id=f"doc-{i % 2}",
                score=1.0 - (i * 0.1),
                text=f"Stub text for {query} ({i})",
                metadata={"source": "stub"},
            )
        )
    return hits

response = SearchResponse(hits=retrieve_stub("example query", top_k=3))
print("hits:", len(response.hits))


Legacy practice content from practice.ipynb

### Task 3.3: Retrieval metrics

Compute simple metrics (count, min/max score) for quick debugging.


In [ ]:
# Legacy practice content
def retrieval_metrics(hits: list[SearchHit]) -> dict:
    if not hits:
        return {"count": 0, "min_score": None, "max_score": None}
    scores = [h.score for h in hits]
    return {
        "count": len(hits),
        "min_score": min(scores),
        "max_score": max(scores),
    }

print("metrics:", retrieval_metrics(response.hits))


Legacy practice content from practice.ipynb

## Self-check

- Are response models explicit and minimal?
- Does retrieval return deterministic results?
- Do metrics make failures visible?
